## MAP-AMP FOR SPIKED-TENSOR-MODELS

This is an implementation of the AMP algorithm for the mixed spiked tensor model (2+3), as written in the MAP-AMP paper (to appear) written in pytorch. While it runs on CPU, porting it on GPU is trivial.

Here is the algorithm, written with a Spherical prior

In [29]:
import torch
from math import sqrt

# Priors
def prior_l2(B):
    """l2 norm prior"""
    N, _ = B.shape
    return B /torch.sqrt(torch.mean(B*B)), 1 /torch.sqrt(torch.mean(B*B))


# AMP Solver
def amp(Y, T, Delta2, Delta3,
        prior=prior_l2, true_coef=None,
        max_iter=250, tol=1e-13, verbose=1):
    """Iterate VAMP equations"""

    N, _ = Y.shape

    # Initialize variables
    B2 = torch.zeros(N, 1)
    B3 = torch.zeros(N, 1)
    a = torch.randn(N, 1)
    a_old = torch.zeros(N, 1)
    c = 1.
    S2 = sqrt(2)
    SN = sqrt(N)
    K = S2 / (Delta3 * N)
    if verbose > 0:
        print("time ; convergence mse, Matrix-mmse ")
    for t in range(max_iter):
        # Messages/estimates on x from likelihood
        B2 = (1. / (SN * Delta2)) * Y @ a - (c / Delta2) * a_old
        B3 = (K * torch.transpose(a, 0, 1)@T@a).view(N, -1)
        B3 = B3 - (2 / (Delta3) ) * c * (torch.mean(a * a_old)) * a_old

        a_old = torch.clone(a)
        a, c = prior_l2(B2 + B3)
        a = 0.5 * a + 0.5 * a_old

        # Compute metrics
        conv = torch.mean((a - a_old) ** 2)
        mse = torch.mean((a - true_coef) ** 2) if true_coef is not None else 0.
        Mmse = torch.mean((a@torch.transpose(a, 0, 1) - (true_coef@torch.transpose(true_coef, 0, 1)))**2) if true_coef is not None else 0.
        if verbose > 0:
            print("t = %d; conv = %g, mse = %g, Mmse = %g" % (t, conv, mse, Mmse))
        if conv < tol:
            break
    return a

Now, this is a short demo on how the algorithm works:

In [31]:
import torch
# We keep CPU by default, if you want GPU uncomment the next line
# torch.set_default_tensor_type('torch.cuda.FloatTensor')#GPU by default
from math import sqrt
import time

N = 300
DELTA2 = 1
DELTA3 = 0.001

X0 = torch.randn(N, 1)
Y = X0@torch.transpose(X0, 0, 1)
T = Y.view(N, N, 1)@torch.transpose(X0, 0, 1)

M_N = torch.randn(N, N)
Matrix_noise = (M_N + torch.transpose(M_N, 0, 1))/sqrt(2)
Y = Y / sqrt(N) + Matrix_noise * sqrt(DELTA2)

T_N = torch.randn(N, N, N)
R1 = torch.transpose(T_N,0,1)
R2 = torch.transpose(T_N,0,2)
R3 = torch.transpose(T_N,1,2)
R4 = torch.transpose(R1,1,2)
R5 = torch.transpose(R2,1,2)
Tensor_noise = (T_N + R1 + R2 + R3 + R4 +R5)/sqrt(6)
T = (sqrt(2) / N) * T + Tensor_noise * sqrt(DELTA3)

t = time.time()
Xhat = amp(Y, T, DELTA2, DELTA3,
           prior=prior_l2,
           true_coef=X0, max_iter=250)
elapsed = time.time() - t

time ; convergence mse, Matrix-mmse 
t = 0; conv = 0.51689, mse = 1.43884, Mmse = 1.1394
t = 1; conv = 0.140289, mse = 1.50407, Mmse = 1.24709
t = 2; conv = 0.0525358, mse = 1.61148, Mmse = 1.43208
t = 3; conv = 0.018514, mse = 1.68808, Mmse = 1.59433
t = 4; conv = 0.00839067, mse = 1.72551, Mmse = 1.70021
t = 5; conv = 0.00585409, mse = 1.72953, Mmse = 1.7581
t = 6; conv = 0.00522691, mse = 1.70173, Mmse = 1.78325
t = 7; conv = 0.00607969, mse = 1.62768, Mmse = 1.77889
t = 8; conv = 0.0128082, mse = 1.44884, Mmse = 1.70882
t = 9; conv = 0.0589981, mse = 0.996467, Mmse = 1.35229
t = 10; conv = 0.153265, mse = 0.383514, Mmse = 0.599224
t = 11; conv = 0.0854827, mse = 0.109552, Mmse = 0.190826
t = 12; conv = 0.0274108, mse = 0.0288594, Mmse = 0.0526954
t = 13; conv = 0.00771606, mse = 0.00804852, Mmse = 0.0149643
t = 14; conv = 0.00208212, mse = 0.00325775, Mmse = 0.00691621
t = 15; conv = 0.000553762, mse = 0.00238984, Mmse = 0.0061315
t = 16; conv = 0.000146759, mse = 0.00236429, Mmse 